In [7]:
import pandas as pd
import re
import pickle
import spacy
import numpy as np
from spacy.lang.en import English

In [ ]:
# Abstract-text-cleaning:leave numbers!
# remove special characters and stop words (delete word indicate tense; creat a new stopword list)
#(note: tense can help with the classification; protocol : future tense ; compeleted RCT: past tense)
#tokenlization and lemma

In [ ]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
nlp = spacy.load('en')

In [ ]:
nlp.Defaults.stop_words -= {"will","had","was","were","did"}

In [ ]:
new_stopwords = spacy_stopwords

In [ ]:
def preprocess_text(df):
    df.Abstract = df.Abstract.fillna(' ')
    df.Abstract = df.Abstract.str.replace('\n',' ')
    df.Abstract = df.Abstract.str.replace('\r',' ')
    return df

In [ ]:
# Correction after checking wrong predictions

In [ ]:
#df_comb['AbL'][6] = 0

In [ ]:
#df_comb['AbL'][382] = 0

C:\anocoda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#df_comb['AbL'][357] = 0

C:\anocoda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# renew df_comb

In [ ]:
df_comb.to_pickle('C:/Users/Combined_df.pkl')

In [ ]:
df_comb = pd.read_pickle('C:/Users/Combined_df.pkl')
#df_comb = df_comb.reset_index(drop =  True)

In [ ]:
#df_comb['AbL'][369]

1

In [ ]:
preprocess_text(df_comb)

,Abstract,AbL
0,Breast cancer survivors (BCSs) often report fa...,0
1,Purpose Cognitive behavioral therapy for insom...,1
2,To investigate the comparative efficacy of cog...,1
3,Ninety-two prenatally depressed pregnant women...,1
4,Fibromyalgia Syndrome (FMS) is characterized b...,1
...,...,...
389,Fibromyalgia is a condition characterised by c...,1
390,Mild cognitive impairment (MCI) is cognitive d...,0
391,There is no sufficient evidence on the effecti...,0
392,Many of the currently available treatments for...,0


In [ ]:
def clean_text(x):
  #x=re.sub(r"\'ll"," will ",x) ----  In academic paper, people tend not to use contraction
  #x=re.sub('[^a-zA-Z0-9]', ' ',x).lower()
  x = re.sub('[^a-zA-Z]', ' ',x).lower()
  x = x.split()
  new_list = []
  for word in x:
    if word not in new_stopwords: 
      new_list.append(word)
  return " ".join(new_list)

In [ ]:
df_comb['Abstract'] = df_comb.Abstract.apply(lambda x:clean_text(x))

In [ ]:
#df_comb[df_comb.index.duplicated(keep = False)]

In [ ]:
X = df_comb.loc[:,'Abstract']

In [ ]:
X

0      breast cancer survivors bcss report fatigue pe...
1      purpose cognitive behavioral therapy insomnia ...
2      investigate comparative efficacy cognitive beh...
3      ninety prenatally depressed pregnant women wer...
4      fibromyalgia syndrome fms characterized muscul...
                             ...                        
389    fibromyalgia condition characterised chronic w...
390    mild cognitive impairment mci cognitive declin...
391    sufficient evidence effectiveness acupuncture ...
392    currently available treatments obsessive compu...
393    chronic fatigue cf unexplained fatigue lasting...
Name: Abstract, Length: 394, dtype: object

In [ ]:
y = df_comb.loc[:,'AbL']

In [ ]:
y

0      0
1      1
2      1
3      1
4      1
      ..
389    1
390    0
391    0
392    0
393    1
Name: AbL, Length: 394, dtype: int64

Train-test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
y_train.value_counts()

1    171
0    144
Name: AbL, dtype: int64

In [ ]:
y_test.value_counts()

1    42
0    37
Name: AbL, dtype: int64

# BOW

In [ ]:
# define max features(words); ngram: to recognize two words as a one, like heart-attack
from sklearn.feature_extraction.text import CountVectorizer 
cv = CountVectorizer(max_features = 5000, ngram_range = (1, 2))
X_train = cv.fit_transform(X_train)
X_test = cv.transform(X_test)
#count vectorizer gives you a sparse matrix: lots of 0s, more efficient for the computer'

In [ ]:
X_train

<315x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 43672 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
classifier = GradientBoostingClassifier(random_state = 0)
classifier.fit(X_train, y_train)

GradientBoostingClassifier(random_state=0)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn import metrics 
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
metrics.auc(fpr, tpr)

0.7648005148005148

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn import metrics 
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
metrics.auc(fpr, tpr)

0.8410553410553411

In [ ]:
# Find the wrong predictions 

In [ ]:
y_pred_series = pd.Series(y_pred, index = y_test.index)

In [ ]:
c = (y_test != y_pred_series)

In [ ]:
diff = c[c].index

In [ ]:
for i in diff:
    print(i, y_test[i],y_pred_series[i], X[i])

74 1 0 nonpharmacologic treatments increasingly adopted alternative primary approaches chronic pain management present results pilot study examining effect week combined abbreviated progressive relaxation technique aprt guided imagery gi intervention management chronic pain n power analysis explore recommended sample sizes future clinical trials results indicated consistent clinically significant trends improvement pain mcgill pain questionnaire visual analog scale mental health depression anxiety stress scale domains quality life rand health survey sleep treatment group owing inadequate power study results were statistically significant methodologic concerns suggestions improved intervention protocol discussed concluded strong preliminary evidence efficacy aprt gi adjunct conventional treatment options chronic pain
135 0 1 post amputation complication phantom limb pain plp prevalent difficult manage study aimed determine was feasible acceptable undertake definitive multicentred random

# TFIDF

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
#fit transform will create inversed document frequency
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(max_features= 4000, ngram_range = (1,2))
X_train = cv.fit_transform(X_train)
X_test = cv.transform(X_test)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
classifier = GradientBoostingClassifier(random_state = 0)
classifier.fit(X_train, y_train)

GradientBoostingClassifier(random_state=0)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn import metrics 
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
metrics.auc(fpr, tpr)

0.7496782496782497

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn import metrics 
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
metrics.auc(fpr, tpr)

0.7599742599742599

#Find wrong predictions and diagnose

In [ ]:
y_pred

array([1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0], dtype=int64)

In [ ]:
y_pred_series = pd.Series(y_pred, index = y_test.index)
y_pred_series

113    1
281    1
68     1
155    1
329    1
      ..
90     0
124    1
64     0
55     1
300    0
Length: 79, dtype: int64

In [ ]:
y_test

113    1
281    1
68     1
155    0
329    1
      ..
90     0
124    1
64     1
55     1
300    0
Name: AbL, Length: 79, dtype: int64

In [ ]:
c = (y_test != y_pred_series)

In [ ]:
c[c].index

Int64Index([155, 357, 229, 334, 342,  74, 135, 164, 308, 361, 316,  76,  52,
            122, 355, 190,  20, 382,  64],
           dtype='int64')

In [ ]:
X[64]

'obesity urgent public health problem clinical trials systematically tested efficacy long term weight loss maintenance interventions randomized clinical trial tested efficacy novel mind body technique weight loss maintenance participants were obese adults had completed month behavioral weight loss program prior randomization successfully lost weight were randomized experimental weight loss maintenance intervention tapas acupressure technique tat control intervention comprised social support group meetings ss led professional facilitators tat combines self applied light pressure specific acupressure points accompanied prescribed sequence mental steps participants maintenance conditions attended group sessions months active weight loss maintenance intervention followed additional 6 months intervention main outcome measure was change weight beginning weight loss maintenance intervention 12 months later secondary outcomes were change depression stress insomnia quality life analysis covaria

In [ ]:
tpr

array([0.        , 0.91111111, 1.        ])

In [ ]:
fpr

array([0.        , 0.44117647, 1.        ])

# BERTs

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.4.1)


In [3]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 135kB/s 
     |████████████████████████████████| 6.8MB 56.2MB/s 
     |████████████████████████████████| 983kB 50.7MB/s 
     |████████████████████████████████| 266kB 47.3MB/s 
     |████████████████████████████████| 1.3MB 42.7MB/s 
     |████████████████████████████████| 1.2MB 49.6MB/s 
     |████████████████████████████████| 471kB 48.7MB/s 
     |████████████████████████████████| 890kB 46.3MB/s 
     |████████████████████████████████| 2.9MB 47.0MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.4-cp36-none-any.whl size=25276443 sha256=3dd5111724c4db2d8a769e2fd887c0df3494e99445f5a1d85a5615e32dfc7803
  Stored in directory: /root/.cache/pip/wheels/1b/77/8a/bdceaabc308e7178d575278bf6143b7d1a9b939a1e40c56b88
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=9f3184c7f9776969c5b0a7e70864737cbe6629659716fd4c05e1d596048c0d45
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [ ]:
#df_valid = df_comb.sample(frac = 0.2, random_state = 42)

In [ ]:
#df_valid.to_pickle('C:/users/Combined_valid.pkl')

In [ ]:
#df_train = df_comb.drop(df_valid.index)

In [ ]:
#df_train.to_pickle('C:/users/Combined_train.pkl')

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [5]:
df_train = pd.read_pickle('/content/drive/MyDrive/Combined_train.pkl')
df_valid = pd.read_pickle('/content/drive/MyDrive/Combined_valid.pkl')

In [6]:
import tensorflow as tf
from sklearn import metrics
import ktrain
from ktrain import text


In [2]:
#print(tf.__version__)

2.4.1


In [7]:
(train, val, preproc) = text.texts_from_df(train_df=df_train, text_column='Abstract', label_columns='AbL',
                                           val_df = df_valid,
                                           maxlen = 512, #length of each doc aligned to 512
                                           preprocess_mode = 'distilbert')


preprocessing train...
language: en
train sequence lengths:
	mean : 179
	95percentile : 294
	99percentile : 579


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 164
	95percentile : 229
	99percentile : 288


In [8]:
bertmodel = text.text_classifier(name = 'distilbert', train_data = train, preproc=preproc)

Is Multi-Label? False
maxlen is 512



done.


In [9]:
learner = ktrain.get_learner(model = bertmodel,
                             train_data = train,
                             val_data = val,
                             batch_size = 6)

In [10]:
learner.fit_onecycle(lr = 2e-5, epochs=10)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
53/53 [==============================] - 29s 397ms/step - loss: 0.6922 - accuracy: 0.5205 - val_loss: 0.6879 - val_accuracy: 0.5316
Epoch 2/10
53/53 [==============================] - 20s 371ms/step - loss: 0.6803 - accuracy: 0.5750 - val_loss: 0.6742 - val_accuracy: 0.5316
Epoch 3/10
53/53 [==============================] - 20s 376ms/step - loss: 0.5951 - accuracy: 0.7082 - val_loss: 0.6406 - val_accuracy: 0.6329
Epoch 4/10
53/53 [==============================] - 20s 381ms/step - loss: 0.5224 - accuracy: 0.7796 - val_loss: 0.5760 - val_accuracy: 0.7215
Epoch 5/10
53/53 [==============================] - 20s 385ms/step - loss: 0.2914 - accuracy: 0.8883 - val_loss: 0.4968 - val_accuracy: 0.7215
Epoch 6/10
53/53 [==============================] - 21s 390ms/step - loss: 0.2450 - accuracy: 0.9146 - val_loss: 0.5574 - val_accuracy: 0.7848
Epoch 7/10
53/53 [==============================] - 21s 394ms/step - loss: 0.16

In [24]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [25]:
text=df_valid['Abstract']
data=list(text)

In [ ]:
#data

In [ ]:
###predictor.get_classes()

In [26]:
y_pred = predictor.predict(data, return_proba=True)

In [27]:
df_valid.AbL = df_valid.AbL.astype('int')
df_train.AbL = df_train.AbL.astype('int')

In [28]:
y_test = df_valid['AbL']

In [29]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred[:,1])
auc = metrics.auc(fpr, tpr)
#plt.plot(fpr, tpr)
auc

0.8622908622908623

In [ ]:
#### Alternative metric reports ####

In [12]:
X_test = df_valid.loc [:,'Abstract']
X_test[78]

'aim study was investigate effects yoga quality life patients cancer patients were yoga program were exercise group years age going treatment cancer were included study physical characteristics patients were recorded general physiotherapy assessments performed sessions classical yoga program including warming breathing exercises asanas relaxation supine position meditation sessions classical exercise program were applied participants yoga exercise program quality life assessments patients were conducted nottingham health profile nhp patients depression levels were assessed beck depression inventory level pain fatigue sleep quality was evaluated visual analog scale vas was found patients quality life scores yoga exercise program were better scores obtained yoga exercise program p post treatment data groups were compared terms nhp subcategories er si s pa total scores nhp were found significantly different favor group p el p scores nhp were different groups p groups were compared terms d

In [13]:
X_test = df_valid.loc [:,'Abstract']

# X_test.keys()
X_test_BERT = []
for keys in X_test.keys():
    X_test_BERT.append(X_test[keys])
#X_test_BERT[0]

In [14]:
y_pred_BERT = predictor.predict(X_test_BERT, return_proba=True)

In [15]:
predictor.save('/content/drive/MyDrive/BERT_HS651CAMs_pred')

In [18]:
y_test = df_valid['AbL']

In [16]:
y_pred_BERT

array([[0.027872  , 0.972128  ],
       [0.01458724, 0.9854128 ],
       [0.0147657 , 0.9852343 ],
       [0.01407404, 0.985926  ],
       [0.02267958, 0.97732043],
       [0.9826984 , 0.01730168],
       [0.99097824, 0.00902181],
       [0.01462934, 0.98537064],
       [0.03579683, 0.9642032 ],
       [0.08695722, 0.9130428 ],
       [0.99222976, 0.00777019],
       [0.9850632 , 0.01493685],
       [0.01577121, 0.98422885],
       [0.92819494, 0.07180506],
       [0.98437554, 0.01562452],
       [0.9912888 , 0.00871122],
       [0.0688794 , 0.93112063],
       [0.13257907, 0.867421  ],
       [0.01889271, 0.98110735],
       [0.9903486 , 0.00965142],
       [0.9872873 , 0.01271269],
       [0.98900175, 0.01099824],
       [0.04100728, 0.9589928 ],
       [0.99095976, 0.00904024],
       [0.9895064 , 0.01049355],
       [0.02203321, 0.9779668 ],
       [0.9102021 , 0.08979792],
       [0.0199863 , 0.9800137 ],
       [0.99038005, 0.00962   ],
       [0.6145859 , 0.3854141 ],
       [0.

In [19]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_BERT[:,1])
auc = metrics.auc(fpr, tpr)
auc

0.8622908622908623

In [ ]:
# Load saved model

In [20]:
saved_predictor = ktrain.load_predictor('/content/drive/MyDrive/BERT_HS651CAMs_pred')

In [21]:
y_pred_BERT1 = saved_predictor.predict(X_test_BERT, return_proba=True)

In [ ]:
y_pred_BERT1 

In [23]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_BERT1[:,1])
auc = metrics.auc(fpr, tpr)
auc

0.8622908622908623